### Creating master index based on features found in filenames

In [1]:
import glob, os, re
from importlib import reload

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

import byc
from byc import steady_state_analysis as ssa
from byc import constants, files, utilities, plotting
for pkg in [constants, files, utilities, plotting]:
    reload(pkg)

patterns = constants.patterns
plotting.set_styles(plt, matplotlib)

### Create master index using features found for each measurement dir and reformat micromanager output so the data can be analyzed using `measure_steady_state` `imagejpc` plugin

In [5]:
reload(files)
reload(constants)

<module 'byc.constants' from 'C:\\Users\\johnp\\Dropbox\\Projects\\byc\\byc\\constants.py'>

In [3]:
files.make_ss_mdf??

In [6]:
exptname = '20230502_PAS1058_autotrophs'
mdf = files.make_ss_mdf(exptname)

Found dataset directory at 
C:\Users\johnp\Box\Finkelstein-Matouschek\images\20230502_PAS1058_autotrophs/data


In [7]:
files.rename_steady_state(master_index_df=mdf)